In [250]:
import pandas as pd
import tabula as tb
import tabula
from PyPDF2 import PdfFileReader
import numpy as np


In [ ]:
#TODO portuguuse commodity code into uk
#TODO export excel file with each pdf as a sheet, using file name as sheet name
#TODO create file paths from folder


In [275]:
paths_pdf = [r"C:\Users\adria\Desktop\Miratoni\Imports\September\10-09-21\Main Lorry Vougafrio\Despachos\Despacho Sal Limiana.pdf"]
paths_csv = [r"C:\Users\\adria\Desktop\Miratoni/Imports/July/23-7-21/Main Lorry/Despacho FAC A21_623 _ 627.csv"]
pd.set_option("max_colwidth", 1000)
pd.set_option("display.max_rows", 200)
codes_path = r"C:\Users\adria\Desktop\Miratoni\Imports\Import Database.csv"

desc_areas = [[146.891,303.45,156.559,574.175],[215.316,303.45,227.216,574.919],[285.228,303.45,296.384,574.175],[355.884,304.194,366.297,574.919],[425.797,303.45,437.697,575.663],[495.709,302.706,506.866,574.919],[566.366,304.194,577.522,574.175],[636.278,303.45,646.691,573.431],[706.191,304.194,716.603,574.919]]
code_areas = [[165.484,304.194,177.384,574.175],[235.397,303.45,246.553,574.175],[306.053,304.194,316.466,574.175],[375.222,303.45,387.866,574.919],[445.878,304.194,456.291,574.919],[516.534,303.45,526.947,575.663],[584.959,303.45,596.116,574.919],[656.359,304.194,666.772,574.919],[726.272,304.194,735.941,574.175]]
gr_wt_areas = [[196.722,490.875,206.391,574.919],[266.634,490.131,277.047,574.175],[335.803,490.875,346.959,574.919],[405.716,489.388,417.616,574.175],[475.628,490.875,486.784,574.175],[545.541,490.875,557.441,574.919],[615.453,490.875,627.353,574.919],[686.109,490.875,697.266,574.175],[756.022,490.875,765.691,574.919]]
net_wt_areas = [[206.391,490.875,216.803,574.175],[275.559,490.875,287.459,574.175],[346.216,490.875,356.628,574.175],[416.128,490.875,426.541,573.431],[486.041,490.875,496.453,574.175],[555.953,490.875,566.366,574.175],[626.609,490.875,636.278,574.919],[695.778,490.875,706.934,573.431],[765.691,490.875,775.359,574.175]]
value_areas = [[205.647,375.594,215.316,491.619],[275.559,375.594,285.972,490.875],[346.216,375.594,355.141,492.363],[416.128,375.594,425.053,492.363],[486.784,375.594,497.197,491.619],[555.953,375.594,566.366,491.619],[625.866,375.594,637.022,491.619],[696.522,375.594,705.447,490.619],[766.434,375.594,776.847,492.363]]
first_page_areas = [[487.528,298.988,499.428,574.175],[509.841,296.756,520.253,574.919],[539.591,487.156,550.003,573.431],[549.259,486.413,560.416,574.175],[549.259,374.85,560.416,486.413]]
normal_areas = [134.991, 303.078, 778.334, 576.778]


In [152]:
def modifier(dffrompdf):
    first_value = [dffrompdf.columns[0]]  #string
    vals = dffrompdf.values.tolist()  #list of lists
    for unit in vals:
        first_value.append(unit[0])
    dffrompdf = first_value
    return dffrompdf    

In [242]:
pdf = PdfFileReader(open(paths_pdf[0],'rb'))
pageno = pdf.getNumPages()
pageno = pdf.getNumPages()
if int(pageno) > 1:
    no_of_pages = f"2-{pageno}"
else:
    no_of_pages = 1    
   

In [184]:
print(no_of_pages)

1


In [243]:
desc = tb.read_pdf(paths_pdf[0], pages=no_of_pages, stream=True, area=desc_areas,
                 multiple_tables=False, format="CSV")
commcode = tb.read_pdf(paths_pdf[0], pages=no_of_pages, stream=True, area=code_areas,
                 multiple_tables=False, format="CSV")
gr_wgt = tb.read_pdf(paths_pdf[0], pages=no_of_pages, stream=True, area=gr_wt_areas,
                 multiple_tables=False, format="CSV")
nt_wgt = tb.read_pdf(paths_pdf[0], pages=no_of_pages, stream=True, area=net_wt_areas,
                 multiple_tables=False, format="CSV")
values = tb.read_pdf(paths_pdf[0], pages=no_of_pages, stream=True, area=value_areas,
                 multiple_tables=False, format="CSV")

all_items = [desc[0], commcode[0], gr_wgt[0], nt_wgt[0], values[0]]

dict_for_df = {'Description':modifier(desc[0]),'PTCode':modifier(commcode[0]),'Gross Weight (kg)':modifier(gr_wgt[0]),'Net Weight(kg)':modifier(nt_wgt[0]),'Value (EUR)':modifier(values[0])}



In [244]:
final = pd.DataFrame.from_dict(dict_for_df)

In [219]:
pg1 = tb.read_pdf(paths_pdf[0], pages=1, stream=True, area=first_page_areas,
                 multiple_tables=False, format="CSV")

pg1_series = pd.Series(modifier(pg1[0]), index = ['Description','PTCode','Gross Weight (kg)','Net Weight (kg)','Value (EUR)'])
pg1_dict = pg1_series.to_dict
pg1_df = pd.DataFrame.from_dict(pg1_series)
pg1_df = pg1_df.transpose()
pg1_df
#pg1_series.to_csv(r'C:\Users\adria\Desktop\Miratoni\Imports\September\10-09-21\Groupagem AM\Despachos\Despacho Cerealis.csv')

,Description,PTCode,Gross Weight (kg),Net Weight (kg),Value (EUR)
0,MASSAS ALIMENTICIAS,19021910.00.0000.0000.0000,1608.44,1410.0,1143


In [222]:
import xlsxwriter
with pd.ExcelWriter('Packing List.xlsx') as writer:
    final.to_excel(writer, sheet_name='Cerealis', index=False)

In [178]:
final.to_csv(r"C:\Users\adria\Desktop\Miratoni\Imports\September\10-09-21\Main Lorry Vougafrio\Despachos\Despacho Sal Limiana.csv")

In [223]:
pg1 = tb.read_pdf(r"C:\Users\adria\Desktop\Miratoni\Imports\September\10-09-21\Groupagem AM\Despachos\Despacho Cerealis.pdf", pages=1, stream=True, area=first_page_areas,
                    multiple_tables=False, format="CSV")
print(pg1)


[          MASSAS ALIMENTICIAS
0  19021910.00.0000.0000.0000
1                     1608.44
2                      1410.0
3                        1143]


In [232]:
pg1_series = pd.Series(modifier(pg1[0]), index = ['Description','PTCode','Gross Weight (kg)','Net Weight (kg)','Value (EUR)'])

pg1_dict = pg1_series.to_dict
pg1_df = pd.DataFrame.from_dict(pg1_series)
final = pg1_df.transpose()


           Description                      PTCode Gross Weight (kg)  \
0  MASSAS ALIMENTICIAS  19021910.00.0000.0000.0000           1608.44   

  Net Weight (kg) Value (EUR)  
0          1410.0        1143  


In [236]:
final['Description']+final['PTCode']

0    MASSAS ALIMENTICIAS19021910.00.0000.0000.0000
dtype: object

In [246]:
final['splitcode'] = final['PTCode'].transform(lambda x: x.replace('.00.0000.0000.0000',''))
final['join'] = final['Description'] + final['PTCode'].transform(lambda x: x.replace('.00.0000.0000.0000',''))
final['join']

0                   BACON02101219
1                 FUMADOS02101989
2                   BANHA15011090
3    ENCHIDOS NAO COZIDOS16010091
4        ENCHIDOS COZIDOS16010099
5         FIAMBRE DE PERU16023119
6                 FIAMBRE16024110
Name: join, dtype: object

In [282]:
codes_df = pd.read_csv(codes_path, dtype=str)
codes_df['join'] = codes_df['Items'] + codes_df['Codes'].transform(lambda x: x[:8])
codes_df

,Items,Codes,join
0,COSTELA DO LOMBO,0203291300,COSTELA DO LOMBO02032913
1,ENTREMEADA,0203291500,ENTREMEADA02032915
2,MIUDOS FRANGO,0207149900,MIUDOS FRANGO02071499
3,PERNIL FUMADO,0210113100,PERNIL FUMADO02101131
4,PERNIL FUMADO,0210113100,PERNIL FUMADO02101131
...,...,...,...
440,FILTROS DE LIQUIDOS,8421298090,FILTROS DE LIQUIDOS84212980
441,APARELHOS P/ CAFE,8516710000,APARELHOS P/ CAFE85167100
442,MOVEIS PLASTICOS,9403700000,MOVEIS PLASTICOS94037000
443,MAQ CAFÉ,8419812090,MAQ CAFÉ84198120


'02032913'